<a href="https://colab.research.google.com/github/Dhruba34/Data-Science-projects/blob/main/assignments%20%26%20project/week%203/task_2_finetune_finbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from transformers import Trainer, TrainingArguments,AutoTokenizer,AutoModelForSequenceClassification
import torch
from datasets import Dataset
import numpy as np
from sklearn.metrics import f1_score,recall_score,precision_score
from sklearn.model_selection import train_test_split


class FinBERTFineTuner:
    def __init__(self, model_name="ProsusAI/finbert"):
        """Initialize for fine-tuning."""
        self.model_name = model_name
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name)
        self.device = "cuda" if torch.cuda.is_available() else "cpu"

    def prepare_dataset(self, texts, labels, max_length=128):
        """
        Prepare texts for training.
        FinBERT expects: [text, label]
        """
        def tokenize_function(examples):
            return self.tokenizer(
                examples['text'],
                padding='max_length',
                truncation=True,
                max_length=max_length
            )

        dataset = Dataset.from_dict({
            'text': texts,
            'label': labels
        })

        tokenized_dataset = dataset.map(
            tokenize_function,
            batched=True,
            remove_columns=['text']
        )

        return tokenized_dataset

    def fine_tune(self, train_texts, train_labels, val_texts, val_labels,
                  num_epochs=3, batch_size=16, learning_rate=2e-5):
        """
        Fine-tune FinBERT on your data.

        Parameters:
        - num_epochs: 3 is typical (more overfits, less undertains)
        - batch_size: 16 for Colab GPU, 8 if memory limited
        - learning_rate: 2e-5 is standard for fine-tuning
        """

        # Prepare datasets
        train_dataset = self.prepare_dataset(train_texts, train_labels)
        val_dataset = self.prepare_dataset(val_texts, val_labels)

        # Define training arguments
        training_args = TrainingArguments(
            output_dir='./finbert_finetuned',
            num_train_epochs=num_epochs,
            per_device_train_batch_size=batch_size,
            per_device_eval_batch_size=batch_size,
            learning_rate=learning_rate,
            warmup_steps=500,
            weight_decay=0.01,
            logging_steps=100,
            eval_strategy="epoch",
            save_strategy="epoch",
            load_best_model_at_end=True,
            metric_for_best_model="f1",
            greater_is_better=True,
            save_total_limit=2,
            report_to=None
        )

        # Define metrics
        def compute_metrics(eval_preds):
            predictions, labels = eval_preds
            predictions = np.argmax(predictions, axis=1)
            accuracy = (predictions == labels).mean()
            precision = precision_score(labels, predictions, average='weighted', zero_division=0)
            recall = recall_score(labels, predictions, average='weighted', zero_division=0)
            f1 = f1_score(labels, predictions, average='weighted', zero_division=0)
            return {
                'accuracy': accuracy,
                'precision': precision,
                'recall': recall,
                'f1': f1
            }

        # Create trainer
        trainer = Trainer(
            model=self.model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=val_dataset,
            compute_metrics=compute_metrics,
        )

        # Train
        print("Starting fine-tuning...")
        trainer.train()
        print("Fine-tuning complete!")

        return trainer

In [ ]:
import pandas as pd

url = "https://raw.githubusercontent.com/Dhruba34/Data-Science-projects/main/assignments%20%26%20project/week%201/all-data.csv"

df = pd.read_csv(
    url,
    engine="python",
    encoding_errors="ignore",
    on_bad_lines="skip",
    names=['sentiment','feedback']
)

print(df.shape)
print(df.head())


(4846, 2)
  sentiment                                           feedback
0   neutral  According to Gran , the company has no plans t...
1   neutral  Technopolis plans to develop in stages an area...
2  negative  The international electronic industry company ...
3  positive  With the new production plant the company woul...
4  positive  According to the company 's updated strategy f...


In [ ]:
labels={'positive':0,'negative':1,'neutral':2}
df['sentiment']=df['sentiment'].apply(lambda x: labels[x])
print(df.head())

   sentiment                                           feedback
0          2  According to Gran , the company has no plans t...
1          2  Technopolis plans to develop in stages an area...
2          1  The international electronic industry company ...
3          0  With the new production plant the company woul...
4          0  According to the company 's updated strategy f...


In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['feedback'].tolist(),
    df['sentiment'].tolist(),
    test_size=0.3,
    random_state=42,
    stratify=df['sentiment']
)


In [ ]:
obj = FinBERTFineTuner()

trainer = obj.fine_tune(
    train_texts=train_texts,
    train_labels=train_labels,
    val_texts=val_texts,
    val_labels=val_labels,
    num_epochs=6,
    learning_rate=1e-6
)


Map:   0%|          | 0/3392 [00:00<?, ? examples/s]

Map:   0%|          | 0/1454 [00:00<?, ? examples/s]

Starting fine-tuning...


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 2


wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Find your API key here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: dhrubajyotipanja-kol (dhrubajyotipanja-kol-iit-bombay) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.275900,0.272108,0.907153,0.909158,0.907153,0.907433
2,0.281700,0.261460,0.909904,0.912126,0.909904,0.910248
3,0.264100,0.256957,0.907840,0.910362,0.907840,0.908313
4,0.244100,0.256606,0.905089,0.907509,0.905089,0.905601
5,0.219600,0.254703,0.905777,0.907894,0.905777,0.906232
6,0.225300,0.256263,0.907840,0.910175,0.907840,0.908293


Fine-tuning complete!


In [ ]:
!wget -O zeroshot.py "https://raw.githubusercontent.com/Dhruba34/Data-Science-projects/main/assignments%20%26%20project/week%203/zeroshot.py"


--2026-01-02 12:25:34--  https://raw.githubusercontent.com/Dhruba34/Data-Science-projects/main/assignments%20%26%20project/week%203/zeroshot.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1882 (1.8K) [text/plain]
Saving to: ‘zeroshot.py’

zeroshot.py         100%[===================>]   1.84K  --.-KB/s    in 0s      

2026-01-02 12:25:35 (31.2 MB/s) - ‘zeroshot.py’ saved [1882/1882]



In [ ]:
import zeroshot

In [ ]:
from sklearn.metrics import accuracy_score

device = "cuda" if torch.cuda.is_available() else "cpu"

# --- Load trained model from fine-tuner ---
trained_model = obj.model  # Already fine-tuned
trained_tokenizer = obj.tokenizer
trained_model.to(device)
trained_model.eval()

# --- Load untrained model from zeroshot.py ---
analyzer = zeroshot.FinBERTAnalyzer()
untrained_model = analyzer.model
untrained_tokenizer = analyzer.tokenizer
untrained_model.to(device)
untrained_model.eval()

# --- Evaluation function ---
def evaluate_model(model, tokenizer, texts, labels):
    preds = []
    for text in texts:
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
        with torch.no_grad():
            outputs = model(**inputs)
            pred = torch.argmax(outputs.logits, dim=1).item()
            preds.append(pred)
    preds = np.array(preds)
    labels = np.array(labels)
    return {
        "accuracy": accuracy_score(labels, preds),
        "precision": precision_score(labels, preds, average="weighted", zero_division=0),
        "recall": recall_score(labels, preds, average="weighted", zero_division=0),
        "f1": f1_score(labels, preds, average="weighted", zero_division=0)
    }

# --- Run evaluation on the full dataset ---
texts = df['feedback'].tolist()
labels = df['sentiment'].tolist()

trained_metrics = evaluate_model(trained_model, trained_tokenizer, texts, labels)
untrained_metrics = evaluate_model(untrained_model, untrained_tokenizer, texts, labels)

print("=== Trained FinBERT Metrics ===")
print(trained_metrics)
print("\n=== Untrained / Zero-shot FinBERT Metrics ===")
print(untrained_metrics)



=== Trained FinBERT Metrics ===
{'accuracy': 0.9160132067684689, 'precision': 0.917839810511113, 'recall': 0.9160132067684689, 'f1': 0.9163835287998549}

=== Untrained / Zero-shot FinBERT Metrics ===
{'accuracy': 0.8893933140734627, 'precision': 0.8994350516214892, 'recall': 0.8893933140734627, 'f1': 0.8906354944517454}
